In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import hddm
import numpy as np
import patsy 

samples = 30000

data = hddm.load_csv('../../../data/1_behav/DataAll.csv')

In [ ]:
def z_link_func(x, data=data):
    return 1 / (1 + np.exp(-(x)))

In [ ]:
# Patch to run PPC
import pymc as pm
import pymc.progressbar as pbar

def _parents_to_random_posterior_sample(bottom_node, pos=None):
    """Walks through parents and sets them to pos sample."""
    for i, parent in enumerate(bottom_node.extended_parents):
        if not isinstance(parent, pm.Node): # Skip non-stochastic nodes
            continue

        if pos is None:
            # Set to random posterior position
            pos = np.random.randint(0, len(parent.trace()))

        assert len(parent.trace()) >= pos, "pos larger than posterior sample size"
        parent.value = parent.trace()[pos]

def _post_pred_generate(bottom_node, samples=500, data=None, append_data=True):
    """Generate posterior predictive data from a single observed node."""
    datasets = []
    ##############################
    # Sample and generate stats
    for sample in range(samples):
        _parents_to_random_posterior_sample(bottom_node)
        # Generate data from bottom node
        sampled_data = bottom_node.random()
        if append_data and data is not None:
            sampled_data.reset_index(inplace=True)  # Only modification of original Kabuki code
            sampled_data = sampled_data.join(data.reset_index(), lsuffix='_sampled')
        datasets.append(sampled_data)
    return datasets

def post_pred_gen(model, groupby=None, samples=500, append_data=False, progress_bar=True):
    results = {}

    # Progress bar
    if progress_bar:
        n_iter = len(model.get_observeds())
        bar = pbar.progress_bar(n_iter)
        bar_iter = 0
    else:
        print("Sampling...")

    if groupby is None:
        iter_data = ((name, model.data.ix[obs['node'].value.index]) for name, obs in model.iter_observeds())
    else:
        iter_data = model.data.groupby(groupby)

    for name, data in iter_data:
        node = model.get_data_nodes(data.index)

        if progress_bar:
            bar_iter += 1
            bar.update(bar_iter)

        if node is None or not hasattr(node, 'random'):
            continue # Skip

        ##############################
        # Sample and generate stats
        datasets = _post_pred_generate(node, samples=samples, data=data, append_data=append_data)
        results[name] = pd.concat(datasets, names=['sample'], keys=list(range(len(datasets))))

    if progress_bar:
        bar_iter += 1
        bar.update(bar_iter)

    return pd.concat(results, names=['node'])

## z & v model

In [ ]:
v_reg = {'model': 'v ~ 1 + stim + condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

zv_model = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

zv_model.sample(samples, burn=samples/10, thin=2)

In [ ]:
zv_model.dic_info['deviance'] + zv_model.dic_info['pD'] + zv_model.dic_info['pD']

In [ ]:
stats = zv_model.gen_stats()
stats.to_csv('../../../data/3_ddm/modeloutputs/subject_parms/zv_model.csv')

In [ ]:
vc, zc = zv_model.nodes_db.node[['v_condition', 'z_condition']]
allParms = vc.trace()
allParms = np.column_stack([allParms,zc.trace()])

np.savetxt("../../../data/3_ddm/modeloutputs/trace_processed/zv_model.csv", allParms , delimiter=",")

In [ ]:
a, t = zv_model.nodes_db.node[['a', 't']]

z_m,z_0 = zv_model.nodes_db.node[['z_condition', 'z_Intercept']] 

v_m,v_s,v_0 = zv_model.nodes_db.node[['v_condition', 'v_stim', 'v_Intercept']] 

allParms = a.trace()
allParms = np.column_stack([allParms,t.trace()])
allParms = np.column_stack([allParms,z_m.trace(),z_0.trace()])
allParms = np.column_stack([allParms,v_m.trace(),v_s.trace(),v_0.trace()])

np.savetxt("../../../data/3_ddm/modeloutputs/trace_processed/zv_model_trace_all.csv", allParms , delimiter=",")

In [ ]:
ppc_data = post_pred_gen(zv_model,samples=100,append_data=True)
np.savetxt("../../../data/3_ddm/modeloutputs/ppc/zv_model_ppc.csv", ppc_data, delimiter=",")

## z model

In [ ]:
v_reg = {'model': 'v ~ 1 + stim', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1 + condition', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

z_model = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

z_model.sa
v model
mple(samples, burn=samples/10, thin=2)

z_model.dic_info['deviance'] + z_model.dic_info['pD'] + z_model.dic_info['pD']

## v model

In [ ]:
v_reg = {'model': 'v ~ 1 + stim + condition', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

v_model = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

v_model.sample(samples, burn=samples/10, thin=2)

v_model.dic_info['deviance'] + v_model.dic_info['pD'] + v_model.dic_info['pD']

## null model

In [ ]:
v_reg = {'model': 'v ~ 1 + stim', 'link_func': lambda x: x}
z_reg = {'model': 'z ~ 1', 'link_func': z_link_func}

reg_descr = [v_reg, z_reg]

null_model = hddm.models.HDDMRegressor(data, reg_descr,
                                       bias=True, p_outlier=0.05,
                                       group_only_regressors=False)

null_model.sample(samples, burn=samples/10, thin=2)

null_model.dic_info['deviance'] + null_model.dic_info['pD'] + null_model.dic_info['pD']